In [ ]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')
from lxml import objectify
import pandas as pd
import os

In [ ]:
os.getcwd()
# os.chdir('xml_testing')

In [ ]:
path='pubmed_result_tighe.xml'
tree = objectify.parse(open(path))

In [ ]:
def stateNcountry(affilation):    
    state = []
    country = []
    city=[]
    for i in affilation:
        words = str(i).split(" ")
        if '@' in words[-1]:
            words.pop()
        if len(words) >= 2:
            country.append(words.pop().rstrip('.').rstrip(',').rstrip(':').rstrip(';'))
            state.append(words.pop().rstrip('.').rstrip(',').rstrip(':').rstrip(';'))
            city.append(words.pop().rstrip('.').rstrip(',').rstrip(':').rstrip(';'))
        if len(words) <2:
            country.append('')
            state.append('')
            city.append('')
    return city,state,country

In [ ]:
author_df = pd.DataFrame(columns=('PMID','LastName','FirstName','City','State', 'Country'))
index = 0

In [ ]:
for i in tree.xpath('//PubmedArticle'):
    pmid = i.xpath(".//PMID")[0]
    lastname = i.xpath(".//Author/LastName")
    lastname = [str(j) for j in lastname]
    firstname = i.xpath(".//Author/ForeName")
    firstname = [str(j) for j in firstname]
    city, state, country = stateNcountry(i.xpath(".//Affiliation"))
    author_tuples = zip(lastname, firstname, city, state, country)
    author_list = [list(j) for j in author_tuples]
    for j in author_list:
        j.insert(0, int(pmid))
        author_df.loc[index] = j
        index += 1
#     return author_df

In [ ]:
author_df[0:10]

This is getting closer, but as you can see the state/country still doesn't always align. It may require some specific post-processing. I've tried to pull a 3rd element out in teh 

Moreover, there seems to be an issue with PMID 26247416 at index=7, where it appears to skip the rest of the authors. I think this is some of the same problems I'm having with trying to pull AbstractText when there are multiple children listed under Abstract, but only sometimes. I've tried to remedy

In [ ]:
del(article_df)

In [ ]:
article_df = pd.DataFrame(columns=('PMID','JournalTitle','ArticleTitle','AbstractText'))
index = 0

In [ ]:
for i in tree.xpath('//PubmedArticle'):
    pmid = i.xpath(".//PMID")[0]
    journaltitle = i.xpath('.//Article/Journal/Title/text()')
    articletitle = i.xpath(".//Article/ArticleTitle/text()")
    abstracttext = i.xpath('.//Abstract/*')
    abstracttext = [str(j) for j in abstracttext]
    ab_tuples = zip(journaltitle, articletitle, abstracttext)
    ab_list = [list(j) for j in ab_tuples]
    for j in ab_list:
        j.insert(0,int(pmid))
        article_df.loc[index] = j
        index += 1

In [ ]:
article_df.loc[1,'AbstractText']

In [ ]:
dfsimple = pd.DataFrame(columns=('PMID','JournalTitle','ArticleTitle','AbstractText'))

In [ ]:
for i in tree.xpath('//PubmedArticle'):
    dfsimple.loc[i,'PMID'] = i.xpath(".//PMID")[0]
#     dfsimple.loc[i, 'JournalTitle'] = i.xpath('.//Article/Journal/Title/text()')
#     dfsimple.loc[i, 'ArticleTitle'] = i.xpath(".//Article/ArticleTitle/text()")
#     abstracttext = i.xpath('.//Abstract/*')

In [ ]:
dfsimple